<a href="https://colab.research.google.com/github/mabeltree/Girish-Fan-Club/blob/main/CNN_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Prepared data based on: https://www.kaggle.com/code/androbomb/using-cnn-to-classify-images-w-pytorch

from torchvision import datasets, transforms

In [ ]:
!git clone https://github.com/mabeltree/Girish-Fan-Club.git

Cloning into 'Girish-Fan-Club'...
remote: Enumerating objects: 2599, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 2599 (delta 6), reused 126 (delta 3), pack-reused 2469
Receiving objects: 100% (2599/2599), 13.90 MiB | 17.13 MiB/s, done.
Resolving deltas: 100% (240/240), done.


In [9]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import image as mp_image
import seaborn as sns

%matplotlib inline

from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

import os

In [10]:
# The images are in a folder named 'input/natural-images/natural_images'
training_folder_name = '../content/Girish-Fan-Club/sneakers'

# All images are 128x128 pixels
img_size = (128,128)

# The folder contains a subfolder for each class of shape
classes = sorted(os.listdir(training_folder_name))
print(classes)


['ALDO', 'AdidasOriginals', 'Aetrex', 'AlexanderMcQueen', 'Amiana', 'BetseyJohnson', 'Converse', 'Crocs', 'DSQUARED2', 'DVS Shoe Company', 'ECCO', 'Easy Spirit', 'GUESS', 'KangaROOS', 'Lacoste', 'Nike', 'Paul Smith', 'Reebok', 'SKECHERS', 'UGG', 'adidasY-3']


In [11]:
# Import PyTorch libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [12]:
from PIL import Image

# function to resize image
def resize_image(src_image, size=(128,128), bg_color="white"):
    from PIL import Image, ImageOps

    # resize the image so the longest dimension matches our target size
    src_image.thumbnail(size, Image.ANTIALIAS)

    # Create a new square background image
    new_image = Image.new("RGB", size, bg_color)

    # Paste the resized image into the center of the square background
    new_image.paste(src_image, (int((size[0] - src_image.size[0]) / 2), int((size[1] - src_image.size[1]) / 2)))

    # return the resized image
    return new_image

In [13]:


# New location for the resized images
train_folder = '../working/data/natural_images'


# Create resized copies of all of the source images
size = (128,128)

# Create the output folder if it doesn't already exist
if os.path.exists(train_folder):
    shutil.rmtree(train_folder)

# Loop through each subfolder in the input folder
print('Transforming images...')
for root, folders, files in os.walk(training_folder_name):
    for sub_folder in folders:
        print('processing folder ' + sub_folder)
        # Create a matching subfolder in the output dir
        saveFolder = os.path.join(train_folder,sub_folder)
        if not os.path.exists(saveFolder):
            os.makedirs(saveFolder)
        # Loop through the files in the subfolder
        file_names = os.listdir(os.path.join(root,sub_folder))
        for file_name in file_names:
            # Open the file
            file_path = os.path.join(root,sub_folder, file_name)
            #print("reading " + file_path)
            image = Image.open(file_path)
            # Create a resized version and save it
            resized_image = resize_image(image, size)
            saveAs = os.path.join(saveFolder, file_name)
            #print("writing " + saveAs)
            resized_image.save(saveAs)

print('Done.')

Transforming images...
processing folder UGG
processing folder DSQUARED2
processing folder DVS Shoe Company
processing folder Nike


<ipython-input-12-d6bfecb2a2da>:8: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  src_image.thumbnail(size, Image.ANTIALIAS)


processing folder ALDO
processing folder Reebok
processing folder Lacoste
processing folder Paul Smith
processing folder Aetrex
processing folder Crocs
processing folder AdidasOriginals
processing folder BetseyJohnson
processing folder KangaROOS
processing folder AlexanderMcQueen
processing folder Amiana
processing folder ECCO
processing folder SKECHERS
processing folder Easy Spirit
processing folder GUESS
processing folder adidasY-3
processing folder Converse
Done.


In [14]:
def load_dataset(data_path):
    import torch
    import torchvision
    import torchvision.transforms as transforms
    # Load all the images
    transformation = transforms.Compose([
        # Randomly augment the image data
            # Random horizontal flip
        transforms.RandomHorizontalFlip(0.5),
            # Random vertical flip
        transforms.RandomVerticalFlip(0.3),
        # transform to tensors
        transforms.ToTensor(),
        # Normalize the pixel values (in R, G, and B channels)
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Load all of the images, transforming them
    full_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=transformation
    )


    # Split into training (70% and testing (30%) datasets)
    train_size = int(0.7 * len(full_dataset))
    test_size = len(full_dataset) - train_size

    # use torch.utils.data.random_split for training/test split
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

    # define a loader for the training data we can iterate through in 50-image batches
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=50,
        num_workers=0,
        shuffle=False
    )

    # define a loader for the testing data we can iterate through in 50-image batches
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=50,
        num_workers=0,
        shuffle=False
    )

    return train_loader, test_loader




#####################################################################################################




# Recall that we have resized the images and saved them into
train_folder = '../working/data/natural_images'

# Get the iterative dataloaders for test and training data
train_loader, test_loader = load_dataset(train_folder)
batch_size = train_loader.batch_size
print("Data loaders ready to read", train_folder)

Data loaders ready to read ../working/data/natural_images


In [ ]:
class Net(nn.Module):


    # Defining the Constructor
    def __init__(self, num_classes=3):
        super(Net, self).__init__()



    def forward(self, x):
        return ...

device = "cpu"
if (torch.cuda.is_available()):
    # if GPU available, use cuda (on a cpu, training will take a considerable length of time!)
    device = "cuda"

# Create an instance of the model class and allocate it to the device
model = Net(num_classes=len(classes)).to(device)

print(model)